# QB-GPT : Using a generalist generative AI model designed for football to better coach defensive plays

<img src="generation.gif" width="1000" style="display: block; margin: 0 auto;" />

## Introduction

Since 2022, generative AI has seen an impressive rise in use, especially in the field of Natural Language Processing and Computer Vision. During this year, I explored how we could use the underlying technology to sports analytics, especially football. This work has led me to release the first generative model for football player trajectories : QB-GPT (for Quarterback Generative Pretrained Transformers). 

QB-GPT is a model using the same technology behing GPT3, called "Attention". It is a model trained to predict a team's trajectory on the field at each 0.3 seconds frame. To do so, the model relies a lot on contextual information closely related to the football game : positions, scrimmage line, type of plays and teammates move. I a way, the model is effectively able to guess how a given lineup is expected to move all together by seeing how this team has done by the past. It is both a model that understands the pysics of the game (speed, cuts, rushs...) and the tactical aspects of playbooks. 

An engine like that can fuel many use cases. I decided to apply the model on the 2024 Big Data Bowl topic and show it can definitely improve coaching by providing an unlimited assistance.

In this section, I will explain my approach leading me to this model. Later I will concretely explain the training and generating component of the model. 

I will conlude on how QB-GPT can help coaching staffs to take brighter decisions during games and the improvements that could be brought to the current technology.

256 words

### Hypothesis

The core hypothesis of QB-GPT is that football is a sequential sports and could be comapred to textual analysis in a way. Games are books. Plays are paragraphs. Players' trajectories are sentences. Players' positions on the field at a given time are words. Those words and sentences (and even paragraphs but I didn't go that far) are not written randomly. It has to make a certain sense which is the tactical spirit of the game. Thus, players move on the pitch according to predefined boundaries and trajectories called "playcalls". 

Starting from this assumption, it is easy to think of a model that generates players' trajectories like GPT generates sentences. The hypothesis is that a model, like a GPT, trained with enough trajectories from enough diverse plays can grasp how a team is expected to move and combine. Then, it can be used to simulate scenarios, predict plays and even recommend trajectories. It is a core engine that compress a lot of football information but needs to have practical use cases to reveal its power. 

174 words

### Data used

I created the first version of QB-GPT using every tracking data available through Kaggle, including the 2024 one. In addition, I got some tracking data available on public repositories listed in appendix. Those sources do not account for more than 10% onf the total data set. In total, my dataste is made of 37,768,611 frames which represent 1,170,109 trajectories over 60,546 plays from 1,042 games. The dataset and related materials are available on request in a parquet format. 

78 words

## What is GB-GPT and how does it work ? 

In this section, I will explain in simple terms how QB-GPT works. It is essential to understand what the model sees, learns and can produce.

### A tool that learns to predict next team movements

As said earlier, football is like a language for our eyes. Players' movements are words. A GPT-3 model learns to predict the next word by looking at all the previous ones. In our case, there is a trick, a player does not move only depending on his previous movements but also based on the ones from all of his teammates. Hence, QB-GPT predicts the team's movements (as one) based on the team precedent moves. 

But what is it predicting ? Instead of predicting the x and y coordinates given by the tracking data, QB-GPT predicts a 1 yard squared zone where the player will be. At the beginning of a play it is very likely to be the zone next to his starting point. But after 3-4 seconds, when players are running fast, the next zone can be a bit far away.

How can it predict correctly ? QB-GPT relies on football knowledge. It does not just see 1 yard squares and guess the next one. For each movement, it knows:
- The time of the play (is it the beginning or the end)
- The play type (is it a pass ? a rush ? a punt ? ...)
- The scrimmage line (are we close to the end zone ? In the middle ? )
- The position of the player for which it predicts (Is it a WR ? An OLB ? A RB ?)
- The starting position of the player (is it close or far away from the scrimmage line)
- The team of the player (Offense or defense ?)

Based on those contextual information, the model is able to adapt its behavior to the reality of the play.

### A tool that can generate simulated plays

Once trained, the model can "generate" plays. What does it mean ? You can provide two lineups, a scrimmage line and a play type and the model will produce a simulated trajectory like a GPT does for words. At each time step (0.3 seconds in our case), the model predict the team position. You can repeat the process iteratively to produce a complete play.



## Better understanding tackles with QG-GPT : How ? 

Predicting tackles or more genrally defensive performance strongly relies on being able to accurately forecast teams' execution for a given play.  Simple use of averages, probabilities or other computed metrics for players or lineups often lack of concrete link with physics of the game. Here is where QB-GPT finds its use case. As QB-GPT is able to generate plays, it is able to represent a very complex distribution of what the play should look like. Drawing several scenarios from this model enables us to explore multiple possibilities as averaging them to obtain an expected play. 

For the task of tackles' prediction, we can simplify it by guessing when two players trajectories will cross. By doing that, we are able to anwser to the thrre questions : Who ? When ? Where ? 
If able to guess those 3 answers, the model can also propose adjustments to make your offensive or defensive lineups even more effective. On the defensive end, it can propose a player that maximizes the probability of tackling a given player. On the offensive end, it can propose a player who whill be the most likely to avoid tackles. 
In addition, it can create trajectories that maximize or minimize the probability of a tackle for a given player.

Using a generalist model always come with flaws. How can we exactly adpat our model to the specificity of a given player, coach or team ? This is where metrics and other indicators come handy as they can put a new perspective to the generated scenarios by the model. In this notebook, I did not have the time to explore cross checks by adding advanced analytics to produce scenarios that are even closer to the reality. I listed it as the first improvement to do.

### QB-GPT can be used to predict tackles per players, their times and locations

Assessing whether a tackle will occur between two players is hard as it strongly relies on the routes taken by each one. The routes of your players is known as they are described int he playbook howver the exectuion can be more or less precise. Furthermore, you have no clue about the routes of your opponents (only some scenarios from your scouting reports). The best you could do with current analysis is having generalistic probabilities more or less accurate for a player and his past performances. Using QB-GPT enables us to overcome those challenges by effectively checking whether two players will have routes that cross (meaning a higher probabilkity of tackle).

The process aims at generating N scenarios from QB-GPT, counting for which players the trajectories are crossing by scenarios and perform an average. We can go further by adding a simple logistic regression which takes the computed probability by the scenarios and some players charachteristics that could be missed by the model (how many successful tackles on the lmast 10 plays ?) and adjust the computed probbaility. At the end, we obtain for each player in the play :
- A probability of tackle by or on every player
- Time for each tackle 
- Location for each tackle

Of course this process strongly relies on the ability of QB-GFPT to generate realistic scenarios based on the teams and lineups. QB-GPT acts as a routes' simulator, a very complex distribution from which we draw several test. However, this process offers also other advantages such as recommendations.

### QB-GPT can build or fill lineups to improve offensive or defensive performance

If we have a model that can generate chnagiung scenarios based on the lineup, the prediction process can be influenced and optimized. For instance, suppose you manually enter the opponent's lineup, now you wish to see what is the lineups that will most effectively attack/or defend against it. In our case, we could say : "What is the lineup that maximizes/minimizes the probability of tackles. This process can be even more precise by selecting a specific player for whom you which to maximize/minimize the probability. Finally, you may have already ideas of lineups (let's say you have 8 players in mind) but unsure about others. QB-GPT can complete the lineup based on the tackling criteria.  

This is possible because QB-GPT is sensitive to changes in lineups, play types and other contextual elements. Thus, it becomes a browser for the team coaching staff. 

### QB-GPT can recommend play calls maximizing/minimizing tackles likelihood (even creating it?)

As for the team part, QB-GPT can be used to create roputes for each players based on your opponents' ones. Let's say you have your lineup, you also are pretty sure of the one from your opponent. Now you could ask QB-GPT what are the possible routes taken by the opponent's team. Based on the simulations of QB-GPT you could parse your playbook and choose the play based on a selected tackle probability. 

# Conclusion, limitations and improvements

In this notebook, I introduced QB-GPT, a generative model for football trained on tracking data of the NFL. The model follows the core principles of a GPT but is significantly smaller (only one attention layer at this stage). This model is deeply rooted in understanding how players move on the field together with some contexutal elements.

QB-GPT can act as a simulator that could be used for a wide variety of task such as tackling prediction and optimization. The prediction part is limited to generate a certain number of scenarios out of two lineups and eventually use a counting methodology to see who tackled who ? Where and when ? The optimization part relies on the ability of the model to change its behavior based on changes in lineups and other elements (play types, scrimmage line...). It can be a powerful tool for coaching staff in order to better select players, scout opponents' playbooks and pick the right play call. This is done by using the generative nature of the model, which enables us to explore possibilites and test hypothesis. 

QB-GPT is limited currently to the data it has seen which may be not fully representative of the diversity of plays, players and situations in a NFL game. The model has been limited to 50 frames of 0.2 seconds, meaning the model is able to generate plays that last until 10 seconds.

QB-GPT has been trained on Google Colab GPU V100 meaning it has some limits in batch size and other hyperparameters. Scaling it to a greatest architecture (A100 or H100) could imapct the quality of the model.

QB-GPT can be improved technically in several manners. On the training part, I would like to conduct a training on a greater scale of data (more plays) to ensure a stabilitiy in the training process. QB-GPT's training can also be upgraded by using latest advancement in the field of GenAI such as sliding attention, flash attention or sink tokens. On a architecture note, it may be a good idea to explore new architectures of models that compete against Generative Pretrained Transformers such as S4 or Liquid Neural Networks.
Operationally, the model could be improved by assessing its latency and speeding up inference (using latest advaancements ). The current state of the QB-GPT project is more a collection of tools and sets that needs to be arranged ans structured into a clear project with a back and front end.

# Appendix

## Bibliography

- 

## Code and related materials

Samuel Chaineau